In [ ]:
import os
import json
import glob
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

In [ ]:
PRE_DIR = "PRE-event"
POST_DIR = "POST-event"
IMG_SIZE = (224, 224)

In [ ]:
def compute_change_features(pre_arr, post_arr):
    pre_arr = pre_arr / 255.0 if pre_arr.max() > 1 else pre_arr
    post_arr = post_arr / 255.0 if post_arr.max() > 1 else post_arr
    diff = np.abs(post_arr - pre_arr)
    return {
        'mean_color_change': np.mean(diff),
        'max_color_change': np.max(diff),
        'std_color_change': np.std(diff),
        'green_decrease': np.mean(post_arr[:,:,1] - pre_arr[:,:,1]),
        'brightness_change': np.mean(np.mean(post_arr, axis=2) - np.mean(pre_arr, axis=2))
    }

In [ ]:
def extract_geojson_features(geojson_path):
    if not os.path.exists(geojson_path):
        return {}
    try:
        with open(geojson_path) as f:
            data = json.load(f)
    except:
        return {}
    features = {'num_buildings': 0, 'num_roads': 0, 'has_building_damage': False}
    for feature in data.get('features', []):
        props = feature.get('properties', {})
        if props.get('building') is not None:
            features['num_buildings'] += 1
        if props.get('highway') is not None:
            features['num_roads'] += 1
        if props.get('flooded') is not None and props.get('flooded') == True:
            features['has_building_damage'] = True
    return features

In [ ]:
print("\n" + "="*70)
print("UNSUPERVISED FLOOD DAMAGE DETECTION PIPELINE")
print("="*70 + "\n")

In [ ]:
print("STEP 1: PREPARING DATASET")
print("-" * 70)

In [ ]:
annotation_files = sorted(glob.glob("annotations/*.geojson"))
print(f"Found {len(annotation_files)} annotation files")
print(f"Processing all files...\n")

In [ ]:
data_list = []

In [ ]:
for idx, anno_file in enumerate(annotation_files):
    tile_name = os.path.basename(anno_file).replace('.geojson', '')
    
    pre_files = glob.glob(f"{PRE_DIR}/*{tile_name}.tif")
    post_files = glob.glob(f"{POST_DIR}/*{tile_name}.tif")
    
    if not pre_files:
        continue
    
    for pre_path in pre_files:
        if post_files:
            post_path = post_files[0]
        else:
            continue
        
        try:
            pre_img = tf.keras.utils.load_img(pre_path, target_size=IMG_SIZE)
            post_img = tf.keras.utils.load_img(post_path, target_size=IMG_SIZE)
            
            pre_arr = tf.keras.utils.img_to_array(pre_img) / 255.0
            post_arr = tf.keras.utils.img_to_array(post_img) / 255.0
        except:
            continue
        
        change_feats = compute_change_features(pre_arr * 255, post_arr * 255)
        geojson_feats = extract_geojson_features(anno_file)
        
        combined_feats = {**change_feats, **geojson_feats}
        combined_feats['pre_filename'] = os.path.basename(pre_path)
        combined_feats['post_filename'] = os.path.basename(post_path)
        combined_feats['tile'] = tile_name
        
        data_list.append(combined_feats)
    
    if (idx + 1) % 50 == 0:
        print(f"  Processed {idx + 1}/{len(annotation_files)} tiles, {len(data_list)} pairs so far...")

In [ ]:
df = pd.DataFrame(data_list)
print(f"\nCreated dataset with {len(df)} image pairs\n")

In [ ]:
if len(df) > 0:
    print("\nSTEP 2: FEATURE STATISTICS")
    print("-" * 70)
    print(df[['mean_color_change', 'max_color_change', 'num_buildings', 'num_roads']].describe())
    
    print("\n\nSTEP 3: FEATURE ENGINEERING & CLUSTERING")
    print("-" * 70)
    
    feature_cols = [col for col in df.columns if col not in ['pre_filename', 'post_filename', 'tile']]
    X = df[feature_cols].fillna(0).values
    
    print(f"Feature matrix shape: {X.shape}")
    print(f"Features used: {feature_cols}")
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    print("\nFeatures normalized (StandardScaler)")
    
    print("\nClustering with KMeans (3 clusters)...")
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    df['cluster'] = clusters
    
    print("\n\nSTEP 4: CLUSTER ANALYSIS")
    print("-" * 70)
    for i in range(3):
        cluster_data = df[df['cluster'] == i]
        print(f"\nCluster {i}: {len(cluster_data)} samples ({100*len(cluster_data)/len(df):.1f}%)")
        print(f"  Color change:  {cluster_data['mean_color_change'].mean():.4f} ± {cluster_data['mean_color_change'].std():.4f}")
        print(f"  Buildings:     {cluster_data['num_buildings'].mean():.1f} ± {cluster_data['num_buildings'].std():.1f}")
        print(f"  Roads:         {cluster_data['num_roads'].mean():.1f} ± {cluster_data['num_roads'].std():.1f}")
        print(f"  Green loss:    {cluster_data['green_decrease'].mean():.4f}")
    
    print("\n\nSTEP 5: SAVING RESULTS")
    print("-" * 70)
    
    df.to_csv('unsupervised_results.csv', index=False)
    print("✓ Saved detailed results to 'unsupervised_results.csv'")
    
    # Create visualization
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    for i in range(3):
        cluster_data = df[df['cluster'] == i]
        plt.scatter(cluster_data['mean_color_change'], cluster_data['num_buildings'], 
                   label=f'Cluster {i}', alpha=0.6, s=50)
    plt.xlabel('Mean Color Change (Damage Indicator)', fontsize=11)
    plt.ylabel('Number of Buildings', fontsize=11)
    plt.title('Flood Damage Clusters', fontsize=12, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    cluster_counts = df['cluster'].value_counts().sort_index()
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    plt.bar(cluster_counts.index, cluster_counts.values, alpha=0.7, color=colors)
    plt.title('Cluster Distribution', fontsize=12, fontweight='bold')
    plt.xlabel('Cluster ID', fontsize=11)
    plt.ylabel('Number of Samples', fontsize=11)
    plt.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('unsupervised_analysis.png', dpi=100, bbox_inches='tight')
    print("✓ Saved visualization to 'unsupervised_analysis.png'")
    
    print("\n" + "="*70)
    print("ANALYSIS COMPLETE!")
    print("="*70 + "\n")
    
    print("OUTPUT FILES:")
    print("  1. unsupervised_results.csv - Full dataset with cluster assignments")
    print("  2. unsupervised_analysis.png - Visualization of clusters")
    print("\nINTERPRETATION GUIDE:")
    print("  - Cluster 0: High color change = Likely flood-damaged areas")
    print("  - Cluster 1: Medium change = Moderate damage/alteration")
    print("  - Cluster 2: Low change = Minimal damage/control areas")
    print("\n")